<a href="https://colab.research.google.com/github/ncn-foreigners/job-ads-classifier/blob/main/job_classification_examples_polski.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## O tutorialu

Notatnik ma na celu przedstawienie sposobu wykorzystania algorytmu klasyfikacji opisów ofert pracy do kodów zawodów określonych w ramach [Klasyfikacji Zawodów i Specjalności 2022 (KSiZ)](https://psz.praca.gov.pl/rynek-pracy/bazy-danych/klasyfikacja-zawodow-i-specjalnosci). W KSiZ zawody określa się na poziomie 1, 2, 3, 4 i 6 cyfrowych kodów. Algorytm tak przygotowany aby można było danej ofercie pracy przypisać kod 6 cyfrowy.

Do uczenia (zbór treningowy i testowy) wykorzystano następujące zbiory danych:

+ oficjalny słownik KSiZ 2022 (tylko treningowy),
+ słownik synonimów GUS dla 6 cyfrowych kodów zawodów (tylko treningowy),
+ słownik [ESCO](https://esco.ec.europa.eu/select-language?destination=/node/1) dla zawodów, które można było przypisać 1-1 (tylko treningowy),
+ zakodowane oferty pracy z [Centralnej Bazy Ofert Pracy](https://oferty.praca.gov.pl/portal/index.cbop) (trening i test),
+ zawody z [Nabory KPRM](https://nabory.kprm.gov.pl/), które można było przypisać 1-1 (trening i test),
+ 3000 ofert z internetu zakodowane przez 3 ekspertki (trening i test).

Uwaga: algorytm przedstawiamy tak jaki jest. Nie gwarantujemy jego 100% poprawości oraz działania na różnych sprzętach. Konfiguracja jest po stronie użytkownika.

Uwaga 2: model oparty na transformerach wymaga do działania GPU.

Autorzy: 
+ Marek Wydmuch (Politechnika Poznańska, OLX; autor kodu i biblioteki napkinXC), 
+ Maciej Beręsewicz (Uniwersytet Ekonomiczny w Poznaniu, Urząd Statystyczny w Poznaniu; zaprojektowanie badania, wymogów i zbiorów), 
+ Herman Cherniaiev (Wyższa Szkoła Informatyki i Zarządzania w Rzeszowie, Instytut Badań Edukcyjnych w Warszawie; przygotowanie danych online, konsultacje),  
+ Robert Pater (Wyższa Szkoła Informatyki i Zarządzania w Rzeszowie, Instytut Badań Edukcyjnych w Warszawie; konsultacje, nadzór).

## Podstaowe ustawienia dla colab

Ustawienia wydruki w colab

In [1]:
from google.colab.data_table import DataTable
DataTable.max_columns = 60

Klonowanie repozytorium

In [2]:
!git clone https://github.com/ncn-foreigners/job-ads-classifier.git 

Cloning into 'job-ads-classifier'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 145 (delta 57), reused 59 (delta 11), pack-reused 0
Receiving objects: 100% (145/145), 3.27 MiB | 16.84 MiB/s, done.
Resolving deltas: 100% (57/57), done.


Środowisko wirtualne na potrzeby projektu

In [3]:
!pip install virtualenv
!virtualenv classifier
!source /content/classifier/bin/activate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 5.1 MB/s 
     |████████████████████████████████| 468 kB 64.6 MB/s 
created virtual environment CPython3.7.15.final.0-64 in 1034ms
  creator CPython3Posix(dest=/content/classifier, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.3, setuptools==65.5.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


Aktualizacja pip.

In [4]:
!python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.8 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


Pobieramy modele z google drive i przerzucamy je w odpowiednie miejsca

In [5]:
!gdown 126YUU3X9AfzdtEJgANzusgo7msonoCzu
!unzip models-v1.1.zip
!rm -rf __MACOSX
!mkdir ./job-ads-classifier/models
!mv transformer_model ./job-ads-classifier/models/
!mv linear_model ./job-ads-classifier/models/

Downloading...
From: https://drive.google.com/uc?id=126YUU3X9AfzdtEJgANzusgo7msonoCzu
To: /content/models-v1.1.zip
100% 2.09G/2.09G [00:18<00:00, 111MB/s]
Archive:  models-v1.1.zip
   creating: linear_model/
  inflating: __MACOSX/._linear_model  
  inflating: linear_model/X_train.bin  
  inflating: __MACOSX/linear_model/._X_train.bin  
  inflating: linear_model/napkinxc_hierarchy.bin  
  inflating: __MACOSX/linear_model/._napkinxc_hierarchy.bin  
  inflating: linear_model/tfidf_vectorizer.bin  
  inflating: __MACOSX/linear_model/._tfidf_vectorizer.bin  
  inflating: linear_model/weights.bin  
  inflating: __MACOSX/linear_model/._weights.bin  
  inflating: linear_model/Y_train.bin  
  inflating: __MACOSX/linear_model/._Y_train.bin  
  inflating: linear_model/stemmer.bin  
  inflating: __MACOSX/linear_model/._stemmer.bin  
  inflating: linear_model/args.bin   
  inflating: __MACOSX/linear_model/._args.bin  
  inflating: linear_model/tree.txt   
  inflating: __MACOSX/linear_model/._tree.t

Instalacja potrzebnych pakietów (uwaga: tu wykorzystujemy plik dla Python 3.7 dla colab). Model był uczony na Python 3.9 i wymaga do działania pliku `requirements.txt`. Po zainstalowaniu należy zrestartować środowisko.

In [6]:
!pip install torch
!pip install -r /content/job-ads-classifier/requirements-py37-colab.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.5/541.5 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.7/361.7 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.1/582.1 kB 38.5 M

## Wykorzystanie istniejących modeli

Podajemy ścieżki do odpowiednich folderów.

In [7]:
classifier_dir="/content/job-ads-classifier"
models_dir="/content/job-ads-classifier/models"

Ładujemy wymagane moduły.

In [8]:
import sys
sys.path.insert(1, '/content/job-ads-classifier')

In [9]:
import numpy as np
import pandas as pd
from job_offers_classifier.load_save import load_to_df, save_obj, load_obj
from job_offers_classifier.job_offers_classfier import *
import torch
from torchmetrics import MetricCollection, Recall, Accuracy, ConfusionMatrix
execfile(f"{classifier_dir}/functions.py")

Pobieramy oferty pracy z nabory KPRM na potrzeby przykładu.

In [10]:
kprm_df = pd.read_xml(f"{classifier_dir}/example-data/kprm-2022-10-21.xml", xpath = "./new/oferta")
kprm_df = kprm_df.replace("\\<\\!\\[CDATA\\[ ", "", regex=True)
kprm_df = kprm_df.replace(" \\]\\]\\>", "", regex=True)
kprm_df = kprm_df.replace(r'<[^<>]*>', ' ', regex=True)
kprm_df = kprm_df.replace('&nbsp;', ' ', regex=True)
kprm_df = kprm_df.replace('&dash;', ' ', regex=True)
kprm_df = kprm_df.replace('&oacute;', ' ', regex=True)
kprm_df = kprm_df.replace('\r', ' ', regex=True)
kprm_df['desc'] = kprm_df[['stanowisko', 'do_spraw', 'nazwa_firmy', 'poledodatkowelista1', 'poledodatkowelista3']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
kprm_df[["unid", "nazwa_firmy", "stanowisko", "do_spraw", "poledodatkowelista1", "poledodatkowelista3"]].head(n=2)

,unid,nazwa_firmy,stanowisko,do_spraw,poledodatkowelista1,poledodatkowelista3
0,108903,Główny Urząd Miar w Warszawie,metrolog,pomiarów w dziedzinie akustyki w Świętokrzyski...,"bierze udział w pracach badawczych, rozwojow...","* MIEJSCE WYKONYWANIA PRACY Warszawa, u..."
1,108902,Ministerstwo Rodziny i Polityki Społecznej w W...,referendarz,współpracy prawnej FGŚP,­Bierze udział w pracach legislacyjnych doty...,Praca administracyjno-biurowa. Opracowy...


Przykładowy opis na potrzeby klasyfikacji.

In [11]:
kprm_df.desc[0]

'metrolog pomiarów w dziedzinie akustyki w Świętokrzyskim Kampusie Laboratoryjnym Głównego Urzędu Miar (ŚKLGUM) Główny Urząd Miar w Warszawie   bierze udział w pracach badawczych, rozwojowych i technicznych związanych z tworzeniem nowych  i doskonaleniem istniejących rozwiązań w zakresie wzorców jednostek miar, metod i stanowisk pomiarowych,  automatyzacji stanowisk pomiarowych w celu potwierdzenia możliwości pomiarowych ŚKLGUM  bierze udział w przygotowaniu i uruchomieniu stanowisk pomiarowych w ŚKLGUM w Kielcach, w tym  w szczególności w opracowywaniu dokumentacji technicznej dla nowotworzonego zespołu stanowisk  pomiarowych z komorą bezechową dużą i małą, udział w szkoleniach w zakresie obsługi aparatury pomiarowej na ww.  stanowiskach, pogłębianie wiedzy specjalistycznej z zakresu obsługi budowanych stanowisk pomiarowych  w celu nabycia zaawansowanej wiedzy i umiejętności niezbędnych do profesjonalnego wykonywania  obowiązków w ŚKLGUM  wykonuje prace związane z utrzymywaniem i dosk

### Wykorzystujemy model regresji logistycznej

Tutaj nazywamy to modelem liniowym.

In [12]:
model_paper_linear = LinearJobOffersClassifier()
model_paper_linear.load(f"{models_dir}/linear_model")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


Predykcja z wykorzystaniem metody `predict`. Argumenty:
+ `X_text` -- opisy zawodów
+ `output_level` -- podajemy poziom KSiZ (0=1 cyfra, 1=2 cyfry, 3=4 cyfry, 5 lub "last" - 6 cyfr)
+ `format` -- podajemy format wynikowy (`dataframe` albo `array`)

In [13]:
kprm_df_zaw1 = model_paper_linear.predict(
    X_text=list(kprm_df["desc"]),
    output_level=0,
    format="dataframe"
)

kprm_df_zaw1.head(n=4)

Processing X ...
Processing text ...


100%|██████████| 450/450 [00:05<00:00, 84.93it/s] 


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 450/450 [00:00<00:00, 1517.00it/s]


,0,1,2,3,4,5,6,7,8,9
0,0.000004,0.000375,0.995414,0.001849,0.000196,0.000036,0.000097,0.001800,1.008496e-04,0.000080
1,0.000036,0.003809,0.975725,0.008887,0.005552,0.001637,0.000095,0.004038,1.148050e-04,0.000061
2,0.000003,0.001122,0.994654,0.000671,0.003392,0.000006,0.000037,0.000016,3.609709e-07,0.000053
3,0.000263,0.055281,0.309090,0.253439,0.157664,0.001704,0.001521,0.211391,1.167186e-03,0.008434


Rozkład zawodów według 1 cyfrowych kodów zawodów.

In [14]:
np.sum(kprm_df_zaw1, axis=0)

0      0.017761
1     13.853405
2    254.529663
3    111.025528
4     53.315598
5      6.214397
6      0.951644
7      3.155583
8      2.712708
9      4.203103
dtype: float32

Możemy również wybrać `top_k` zawodów. Poniżej wskazujemy top 5.

In [15]:
kprm_df_zaw6_top5 = model_paper_linear.predict(
    X_text=list(kprm_df["desc"]),
    output_level="last",
    format="dataframe",
    top_k=5
)

Processing X ...
Processing text ...


100%|██████████| 450/450 [00:02<00:00, 189.99it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 450/450 [00:00<00:00, 1587.10it/s]


In [16]:
kprm_df_zaw6_top5.head(n=5)

,class_1,class_2,class_3,class_4,class_5,prob_1,prob_2,prob_3,prob_4,prob_5
0,214921,214922,214990,214903,214932,0.635320,0.203939,0.019870,0.012856,0.009343
1,242217,242190,242102,242290,242232,0.818859,0.063468,0.017318,0.011158,0.006835
2,242217,241107,241190,241103,241102,0.697043,0.109484,0.087459,0.032341,0.012461
3,242217,711101,335290,411003,334306,0.271974,0.106116,0.072340,0.069055,0.061913
4,325512,335990,325502,335905,334306,0.269934,0.133360,0.103502,0.067749,0.048600


### Transformer model

Ten model będzie działał tylko jak będzie dostęp do GPU na colab. Bez GPU pojawia się problem z wczytaniem danych na Colab.

In [ ]:
model_paper_trans = TransformerJobOffersClassifier(gpus=0, threads=2) ## bez gpu
#model_paper_trans = TransformerJobOffersClassifier(gpus=1, threads=2)
model_paper_trans.load(f"{models_dir}/transformer_model")

Initializing TransformerClassifier with model_name=allegro/herbert-base-cased, output_type=linear, num_labels=2904, learning_rate=1e-05, weight_decay=0.01, warmup_steps=50 ...


Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624M [00:00<?, ?B/s]

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.sso.sso_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.sso.sso_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Predykcja z wykorzystaniem modelu opartego na transformerach. Uwaga: teraz podajemy argument `X`, a nie `X_text`. Reszta pozostaje bez zmian.

In [ ]:
kprm_df_zaw1_trans = model_paper_trans.predict(
    X=list(kprm_df["desc"]),
    output_level=0,
    format="dataframe"
)

Initializing TransformerDataModule with model_name=allegro/herbert-base-cased, max_seq_length=512, train/eval_batch_size=8/8, num_workers=2 ...
Setting up TransformerDataModule ...


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:667: UserWarning: You passed `Trainer(accelerator='cpu', precision=16)` but native AMP is not supported on CPU. Using `precision='bf16'` instead.
  "You passed `Trainer(accelerator='cpu', precision=16)` but native AMP is not supported on CPU."
INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:p

Starting predicting with TransformerClassifier ...


INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Predicting: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:727: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


TypeError: ignored

Rozkład 1 cyfrowych kodów zawodów.

In [ ]:
np.sum(kprm_df_zaw1_trans, axis=0)

NameError: ignored

Wybieramy top 5 zawodów.

In [ ]:
kprm_df_zaw6_top5_trans = model_paper_trans.predict(
    X=list(kprm_df["desc"]),
    output_level="last",
    format="dataframe",
    top_k=5
)

Initializing TransformerDataModule with model_name=allegro/herbert-base-cased, max_seq_length=512, train/eval_batch_size=8/8, num_workers=2 ...
Setting up TransformerDataModule ...


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Starting predicting with TransformerClassifier ...


INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Predicting: 0it [00:00, ?it/s]

In [ ]:
kprm_df_zaw6_top5_trans.head(n=5)

NameError: ignored

Porównujemy wybrane wyniki dla tych ofert, w których nie ma zgodności między rozważanymi modelami.

In [ ]:
kprm_df["lin"] = kprm_df_zaw6_top5["class_1"]
kprm_df["trans"] = kprm_df_zaw6_top5_trans["class_1"]
kprm_df[kprm_df["lin"].str.slice(0,1) != kprm_df["trans"].str.slice(0,1)][
    ["trans", "lin", "stanowisko", "do_spraw", "desc"]].head(n=5)

,trans,lin,stanowisko,do_spraw,desc
3,242307,441590,referent,ds. kadr i płac,referent ds. kadr i płac Powiatowy Inspektorat...
12,242204,132102,audytor wewnętrzny,None,audytor wewnętrzny None Ministerstwo Spraw Zag...
18,411003,341103,podreferendarz,obsługi kancelaryjnej,podreferendarz obsługi kancelaryjnej Ministers...
21,311104,216590,inspektor wojewódzki,państwowego zasobu geodezyjnego i kartograficz...,inspektor wojewódzki państwowego zasobu geodez...
25,411003,242217,referent,sekretariatu,referent sekretariatu Izba Administracji Skarb...


Ogólna zgodność wygląda następująco

## Jakość modelu

Poniżej przedstawiono jakość modelu na podstawie ofert pracy kodowanych ręcznie. 

In [233]:
labeled = pd.read_csv("https://github.com/ncn-foreigners/job-ads-datasets/raw/main/data/hand-coded.csv", dtype="str")
labeled["code"]=labeled["code"].str.strip()
labeled["desc"]=labeled["desc"].str.strip()
labeled = labeled[~pd.isna(labeled["code"])].drop_duplicates().reset_index()
labeled2 = labeled.groupby("desc")['code'].apply(lambda x: list(x)).reset_index()

In [234]:
labeled_code6 = model_paper_linear.predict(
    X_text=list(labeled2["desc"]),
    output_level="last",
    format="dataframe",
    top_k=3
)

Processing X ...
Processing text ...


100%|██████████| 9536/9536 [00:46<00:00, 204.16it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 9536/9536 [00:05<00:00, 1643.29it/s]


In [235]:
labeled2["predictions"]= labeled_code6[["class_1", "class_2", "class_3"]].apply(lambda x: [str(x.class_1), str(x.class_2), str(x.class_3)], axis = 1)
labeled2["pred1_overlap"]=labeled2.apply(lambda x: len([val for val in [i[0] for i in x.predictions[0:len(x.code)]] if val in [str(j)[0] for j in x.code]])/len(x.code), axis = 1)
labeled2["pred2_overlap"]=labeled2.apply(lambda x: len([val for val in [i[0:2] for i in x.predictions[0:len(x.code)]] if val in [str(j)[0:2] for j in x.code]])/len(x.code), axis = 1)
labeled2["pred4_overlap"]=labeled2.apply(lambda x: len([val for val in [i[0:5] for i in x.predictions[0:len(x.code)]] if val in [str(j)[0:5] for j in x.code]])/len(x.code), axis = 1)
labeled2["pred6_overlap"]=labeled2.apply(lambda x: len([val for val in x.predictions[0:len(x.code)] if val in x.code])/len(x.code), axis = 1)

Porównanie jakości predykcji na 1, 2, 4 i cyfrach uwzględniając możliwość wielu kodów.

In [236]:
[np.sum(labeled2["pred1_overlap"]) / labeled2.shape[0],
 np.sum(labeled2["pred2_overlap"]) / labeled2.shape[0],
 np.sum(labeled2["pred4_overlap"]) / labeled2.shape[0],
 np.sum(labeled2["pred6_overlap"]) / labeled2.shape[0]]

[0.8583962527964205, 0.8383144574944071, 0.7801489093959733, 0.764104446308725]

Porównanie predykcji dla 1 kodu według 1 cyfrowych kodów

In [237]:
labeled2["code_2_true"]=labeled2["code"].apply(lambda x: str(x[0])[0])
labeled2["code_2_pred"]=labeled2["predictions"].apply(lambda x: str(x[0])[0])

Predykcja jest najgorsza dla kodów 3 (72%) i 1 (77%), a najlepsza dla 2 (91%) i 5 (90%).

In [245]:
pd.crosstab(index = labeled2["code_2_true"], 
            columns = labeled2["code_2_pred"])

code_2_pred,1,2,3,4,5,6,7,8,9
code_2_true,,,,,,,,,
1,420,81,20,4,12,0,2,1,3
2,48,2728,118,47,38,0,17,1,2
3,12,155,924,51,74,0,43,13,3
4,6,57,41,646,14,0,4,6,18
5,8,55,27,14,1028,0,6,0,6
6,0,1,0,0,0,16,0,0,1
7,6,19,31,2,5,0,1268,32,73
8,1,2,6,14,5,0,42,587,19
9,1,4,7,28,20,2,42,13,536


Przykladowa zła predykcja gdzie koordynator ds. logistyki jest klasyfikowany jako kierowca.

In [262]:
labeled2[(labeled2["code_2_true"]=="2") & (labeled2["code_2_pred"]== "8")].iloc[0,0]

'Koordynator ds. logistyki ZAKŁADY PRODUKCJI SPOŻYWCZEJ AMBI M KARKUT I WSPÓLNICY SPÓŁKA JAWNA kierownik / koordynator Łańcuch dostaw Twój zakres obowiązków Nadzór nad flotą samochodów osobowych i dostawczych Odpowiedzialność za planowanie i organizowanie pracy kierowców na dzień następny (20-25 pracowników)Szczegółowe rozliczanie kierowców z dnia pracy Odpowiedzialność za jakość i terminowość dostaw do klientów Bieżąca kontrola zużycia paliwa oraz optymalizowanie tras kierowców - GPS Codzienny monitoring stanu technicznego taboru, weryfikacja zgłoszeń kierowców oraz zakresu zleconych i wykonanych prac serwisowych Prowadzenie cyklicznych szkoleń kierowców Raportowanie Nasze wymagania wykształcenie min. średnie umiejętność kierowania zespołem ludzkim podstawowa wiedza z zakresu czasu pracy kierowców umiejętność podejmowania decyzji pod presją czasu dobra organizacja pracy To oferujemy zatrudnienie na podstawie umowy o pracę lub umowę o współpracy wszelkie niezbędne narzędzia do wykonywa

Ten przypadek może być jednak błędem poniewaz kod nadany ręcznie to `912990` (Pozostali pracownicy zajmujący się sprzątaniem gdzie indziej niesklasyfikowani), a kody `143907` (Kierownik firmy sprzątającej) i `143990` (Pozostali kierownicy do spraw innych typów usług gdzie indziej niesklasyfikowani) z czego drugi jest możliwy ale pierwszy już nie.


In [260]:
labeled2[(labeled2["code_2_true"]=="9") & (labeled2["code_2_pred"]== "1")].iloc[0,0]

'Koordynator zespołu sprzątającego (k/m) STOKSON Spółka Jawna Henryk Stokłosa i Wspólnicy   Kto szuka: STOKSON Spółka Jawna Henryk Stokłosa i Wspólnicy Stanowisko: Koordynator zespołu sprzątającego (k/m) Lokalizacja: Piekary Śląskie śląskie Opis stanowiska podany przez pracodawcę: Opis stanowiska: Prace porządkowe oraz nadzór nad pracownikami sprzątającymi, Kontrola zużycia oraz zamawianie środków czystości, Nadzór nad realizacją prac, Aktywne uczestnictwo w pracy zespołu, Raportowanie do przełożonego. Wymagania stawiane pracownikowi: Wymagania: Doświadczenie zawodowe na podobnym stanowisku, Umiejętność kierowania zespołem, Znajomość pakietu MS Office, Dyspozycyjność do pracy w systemie zmianowym. Firma oferuje: Oferujemy: Stabilną i satysfakcjonującą pracę w oparciu o umowę o pracę na pełen etat, Atrakcyjne wynagrodzenie, Pracę w prężnie rozwijającej się firmie, Wysoki standard pracy, Możliwość przystąpienia do ubezpieczenia grupowego, System kafeteryjny - do wyboru m.in. zajęcia spor